# Imports

In [1]:
import pandas as pd
import numpy as np

# constants

In [2]:
TRAIN_DATA = 'C:/Users/USER/spam/SpamData/01_Processing/training-data.txt'
TEST_DATA_FILE = 'C:/Users/USER/spam/SpamData/01_Processing/test-data.txt'
VOCAB_SIZE = 2500

# Read and Load Features from .txt files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAIN_DATA, delimiter=' ',dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ',dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
sparse_test_data[-5:]

array([[5793, 2329,    0,    2],
       [5793, 2353,    0,    1],
       [5793, 2374,    0,    4],
       [5793, 2403,    0,    3],
       [5793, 2491,    0,    1]])

In [7]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 258360
Nr of rows in test file 117776


In [8]:
print('No of email in training file', np.unique(sparse_train_data[:,0]).size)
print('No of email in test file', np.unique(sparse_test_data[:,0]).size)

No of email in training file 4013
No of email in test file 1724


## How to create an empty DataFrame

In [9]:
column_names= ['DOC_ID']+['CATEGORY']+ list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [10]:
len(column_names)

2502

In [11]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [12]:
full_train_data = pd.DataFrame(index= index_names, columns = column_names)
full_train_data.fillna(value=0, inplace = True)

full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a full Matrix from Sparse Matrix

In [13]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    column_names= ['DOC_ID']+['CATEGORY']+ list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_train_data[:,0])
    full_matrix = pd.DataFrame(index=index_names, columns = column_names)
    full_matrix.fillna(value=0, inplace = True)
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurance = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr,'DOC_ID']= doc_nr
        full_matrix.at[doc_nr,'CATEGORY']= label
        full_matrix.at[doc_nr, word_id] = occurance
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix


In [14]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 6.95 s


In [15]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training the naive Bayes Model
## Calculating the Probability of Spam

In [19]:
full_train_data.CATEGORY.size

4013

In [17]:
full_train_data.CATEGORY.sum()

1248

In [21]:
prob_spam= full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
print('Probability of spam is=',prob_spam)

Probability of spam is= 0.310989284824321


# Total Number of Words/Tokens

In [23]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATRGORY']
full_train_features.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [25]:
email_lengths[:5]

DOC_ID
0     88
1     54
2     41
3    184
4     44
dtype: int64

In [27]:
total_wc = email_lengths.sum()
total_wc

430488

# No of tokens in Spam and Non Spam Emails

In [29]:
spam_lengths = email_lengths[full_train_data.CATEGORY ==1]
spam_lengths.shape

(1248,)

In [30]:
spam_wc = spam_lengths.sum()
spam_wc

177581

In [35]:
non_spam_lengths = email_lengths[full_train_data.CATEGORY ==0]
non_spam_lengths.shape

(2765,)

In [37]:
spam_lengths.shape[0] - spam_lengths.shape[0] - non_spam_lengths.shape[0]

-2765

In [41]:
nonspam_wc = spam_lengths.sum()
nonspam_wc

177581

In [55]:
print('Average no of words in spam emails {:.0f}'.format(spam_wc/spam_lengths.shape[0]))

Average no of words in spam emails 142


In [58]:
print('Average no of words in non spam emails {:0.2f}'.format(nonspam_wc/non_spam_lengths.shape[0]))

Average no of words in non spam emails 64.22


# Summing the tokens Occuring in spam

In [59]:
full_train_features.shape

(4013, 2501)

In [63]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
train_spam_tokens.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,1,0,0,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1890,1,2,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
train_spam_tokens.shape

(1248, 2501)

In [68]:
summed_spam_tokens = train_spam_tokens.sum(axis = 0) +1
summed_spam_tokens.shape

(2501,)

In [69]:
summed_spam_tokens.tail()

2495     3
2496     2
2497    17
2498    15
2499     2
dtype: int64

# Summing the Tokens Occuring in Non-spam

In [72]:
train_non_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]

In [73]:
summed_non_spam_tokens = train_non_spam_tokens.sum(axis = 0)+1

In [75]:
summed_non_spam_tokens.tail()

2495    27
2496    35
2497    20
2498    15
2499    28
dtype: int64

In [76]:
train_non_spam_tokens[2499].sum() +1

28

## P(Token| Spam) - Probability that Token Occurs given the Email is Spam.

In [78]:
prob_tokens_spam = summed_spam_tokens /(spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

CATEGORY    0.006936
0           0.012100
1           0.005192
2           0.006758
3           0.011228
dtype: float64

In [79]:
prob_tokens_spam.sum()

1.00000555305668

## P(Token| Non-Spam) - Probability that Token Occurs given the Email is Non-Spam.

In [80]:
prob_tokens_nonspam = summed_non_spam_tokens/(nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.4182951005380913

# P(Token) Probability that Token Occurs

In [81]:
prob_tokens_all = full_train_features.sum(axis = 0)/ total_wc

In [82]:
prob_tokens_all.sum()

1.0